In [45]:

from pathlib import Path

def _start_or_end_with(text, pattern):
  pattern = tuple(pattern)
  return text.startswith(pattern) or text.endswith(pattern)

def load_csv(path, sample=10, only_title=False, include=(), exclude=()):
  from itertools import compress
  import csv

  with open(path) as f:
    titles = f.readline().strip().split(',')

  if include:
    column_compress = [_start_or_end_with(title, include) for title in titles]
  else:
    column_compress = [True] * len(titles)
  if exclude:
    column_compress = [not _start_or_end_with(title, exclude) and tb for title, tb in zip(titles, column_compress)]

  if only_title:
    return list(compress(titles, column_compress))

  with open(path) as f:
    lines = csv.reader(f)
    next(lines)
    result = []
    for i, line in enumerate(lines, 1):
      if sample and i > sample:
        break
      # result.append([_load_csv_value_convert(x) for x in compress(line, column_compress)])
      result.append([x for x in compress(line, column_compress)])

  return result



    目录结构大致是
    .
    |____cancer mortality white male 65 - sheet1.csv
    |____ewg_process.ipynb
    |____epa_water_system
    | |____epa_water_system_summary_AK_[3549].csv
    | |____epa_water_system_summary_AL_[1810].csv
    | |____epa_water_system_summary_AR_[4194].csv
    | |____epa_water_system_summary_AS_[273].csv
    | |____epa_water_system_summary_AZ_[3967].csv
    | |____epa_water_system_summary_CA_[18131].csv
    | |____epa_water_system_summary_CO_[4588].csv
    | |____epa_water_system_summary_CT_[11043].csv
    | |____epa_water_system_summary_DE_[1300].csv
    | |____......
    | |____......
    | |____......
    | |____epa_water_system_summary_TN_[4939].csv
    | |____epa_water_system_summary_TX_[15282].csv
    | |____epa_water_system_summary_UT_[2114].csv
    | |____epa_water_system_summary_VA_[8721].csv
    | |____epa_water_system_summary_VI_[986].csv
    | |____epa_water_system_summary_VT_[4286].csv
    | |____epa_water_system_summary_WA_[9465].csv
    | |____epa_water_system_summary_WI_[23678].csv
    | |____epa_water_system_summary_WV_[4337].csv
    | |____epa_water_system_summary_WY_[1859].csv
    | 
    |____ewg_all_done[22466]_nodata[396908].csv



### 找出 `cancer mortality white male 65.xls` 中所有需要处理的 州-县

In [40]:
sheet = './cancer mortality white male 65 - sheet1.csv'
headers = load_csv(sheet, only_title=True)
print('headers: ', headers)


county_list = load_csv(sheet, sample=None, include=('County', ))

county_list = [elem[0].split(' (')[0] for elem in county_list if elem[0] != 'United States (6)']

print('\n\ncounties: ', county_list[:20], '...')

headers:  ['County', ' FIPS', 'Met Healthy People Objective of 161.4?', '"Age-Adjusted Death Rate(? - deaths per 100', '000"', 'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval', 'Average Annual Count', 'Recent Trend', 'Recent 5-Year Trend (? in Death Rates', 'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval', '', 'life expenctancy', '']


counties:  ['Douglas County, Colorado', 'Fairfax County, Virginia', 'Marin County, California', 'Santa Clara County, California', 'Loudoun County, Virginia', 'Collier County, Florida', 'Howard County, Maryland', 'San Mateo County, California', 'Bergen County, New Jersey', 'Olmsted County, Minnesota', 'Stearns County, Minnesota', 'Westchester County, New York', 'Hunterdon County, New Jersey', 'Orange County, California', 'Rockland County, New York', 'Nassau County, New York', 'Morris County, New Jersey', 'Somerset County, New Jersey', 'Washington County, Minnesota', 'Ozaukee County, Wisconsin'] ...


### 找出 `epa_water_system/` 中对应 州-县 的所有水厂 ID

In [49]:
state_dict = '''阿拉巴马州	Alabama	AL	蒙哥马利	Montgomery
阿拉斯加州	Alaska	AK	朱诺	Juneau
阿利桑那州	Arizona	AZ	菲尼克斯	Phoenix
阿肯色州	Arkansas	AR	小石城	Little rock
加利福尼亚州	California	CA	萨克拉门托	Sacramento
科罗拉多州	Colorado	CO	丹佛	Denver
康涅狄格州	Connecticut	CT	哈特福德	Hartford
特拉华州	Delaware	DE	多佛	Dover
佛罗里达州	Florida	FL	塔拉哈西	Tallahassee
乔治亚州	Georgia	GA	亚特兰大	Atlanta
夏威夷州	Hawaii	HI	檀香山www.fltacn.com火努鲁鲁	Honolulu
爱达荷州	Idaho	ID	博伊西	Boise
伊利诺斯州	Illinois	IL	斯普林菲尔德	Springfield
印第安纳州	Indiana	IN	印第安纳波利斯	Indianapolis
爱荷华州	Iowa	IA	得梅因	Des Moines
堪萨斯州	Kansas	KS	托皮卡	Topeka
肯塔基州	Kentucky	KY	法兰克福	Frankfort
路易斯安那州	Louisiana	LA	巴吞鲁日	Baton Rouge
缅因州	Maine	ME	奥古斯塔	Augusta
马里兰州	Maryland	MD	安纳波利斯	Annapolis
马萨诸塞州	Massachusetts	MA	波士顿	Boston
密歇根州	Michigan	MI	兰辛	Lansing
明尼苏达州	Minnesota	MN	圣保罗	St. Paul
密西西比州	Mississippi	MS	杰克逊	Jackson
密苏里州	Missouri	MO	杰斐逊城	Jefferson City
蒙大拿州	Montana	MT	海伦娜	Helena
内布拉斯加州	Nebraska	NE	林肯	Lincoln
内华达州	Nevada	NV	卡森城	Carson City
新罕布什尔州	New hampshire	NH	康科德	Concord
新泽西州	New jersey	NJ	特伦顿	Trenton
新墨西哥州	New mexico	NM	圣菲	Santa Fe
纽约州	New York	NY	奥尔巴尼	Albany
北卡罗来纳州	North Carolina	NC	纳罗利	Raleigh
北达科他州	North Dakota	ND	俾斯麦	Bismarck
俄亥俄州	Ohio	OH	哥伦布	Columbus
俄克拉荷马州	Oklahoma	OK	俄克拉何马城	Oklahoma City
俄勒冈州	Oregon	OR	塞勒姆	Salem
宾夕法尼亚州	Pennsylvania	PA	哈里斯堡	Harrisburg
罗得岛州	Rhode island	RI	普罗维登斯	Providence
南卡罗来纳州	South carolina	SC	哥伦比亚	Columbia
南达科他州	South dakota	SD	皮尔	Pierre
田纳西州	Tennessee	TN	纳什维尔	Nashville
得克萨斯州	Texas	TX	奥斯汀	Austin
犹他州	Utah	UT	盐湖城	Salt Lake City
佛蒙特州	Vermont	VT	蒙彼利埃	Montpelier
弗吉尼亚州	Virginia	VA	里士满	Richmond
华盛顿州	Washington	WA	奥林匹亚	Olympia
西弗吉尼亚州	West Virginia	WV	查尔斯顿	Charleston
威斯康辛州	Wisconsin	WI	麦迪逊	Madison
怀俄明州	Wyoming	WY	夏延	Cheyenne'''

state_dict = {line.split('\t')[1]: line.split('\t')[2] for line in state_dict.splitlines()}




def find_epa_water_system_csv_filename(state_county_name):
  state_name = state_county_name.split(',')[1].strip()
  state_alias = state_dict[state_name]
#   filename like epa_water_system_summary_OR_[5802].csv
  ret = list(Path('./epa_water_system/').glob('epa_water_system_summary_{}*.csv'.format(state_alias)))
  if len(ret) != 1:
    raise ValueError
  else:
    return str(ret[0])
  
  


def find_pwsids_from_epa_water_system_csv(state_county_name, epa_water_system_csv):
  county_name = state_county_name.split(',')[0].strip().replace(' County', '')
  print(county_name)
  epa_data = load_csv(epa_water_system_csv, sample=None, include='PWS ID,Counties Served,Population Served Count'.split(','))
  return [(pwsid, int(pop_count.replace(',', ''))) for pwsid, name, pop_count in epa_data if name == county_name]




state_county_name = 'Douglas County, Colorado'
state_county_name = 'Adams County, Colorado'
# state_county_name = 'Stearns County, Minnesota'

epa_water_system_csv = find_epa_water_system_csv_filename(state_county_name)
print('find epa_water_system_csv', epa_water_system_csv)

county_pwsid_and_pop_count_list = find_pwsids_from_epa_water_system_csv(state_county_name, epa_water_system_csv)
print(county_pwsid_and_pop_count_list[:20], '...', len(county_pwsid_and_pop_count_list))

find epa_water_system_csv epa_water_system/epa_water_system_summary_CO_[4588].csv
Adams
[('CO0101001', 0), ('CO0053876', 200), ('CO0101005', 0), ('CO0101010', 0), ('CO0201006', 257), ('CO0028969', 85), ('CO0101015', 460), ('CO0101020', 3100), ('CO0116140', 3396), ('CO0117358', 101), ('CO0101025', 36010), ('CO0101030', 77), ('CO0101166', 150), ('CO0101035', 0), ('CO0101185', 1000), ('CO0101188', 160), ('CO0101040', 17000), ('CO0101187', 100), ('CO0101205', 1200), ('CO0101234', 1193)] ... 165


### 用对应 州-县 的所有水厂 ID 找到 Nitrate 数据, 大部分应该是没有数据

In [47]:
nitrate_data = {}


# 计算一次就行, 很慢
for pwsid, state, national, this_utility in load_csv('./ewg_all_done[22466]_nodata[396908].csv', sample=None):
  if this_utility != 'nodata':
    nitrate_data[pwsid] = float(this_utility.replace(' ppm', ''))

len(nitrate_data.keys())

22466

### 对有数据的部分做加权平均, 得到该 州-县 的 Nitrate 水平

In [50]:

county_nitrate_list = [(pop_count, nitrate_data[pwsid]) for pwsid, pop_count in county_pwsid_and_pop_count_list if pwsid in nitrate_data]


print(county_nitrate_list)

print()
print(state_county_name, '- mean_nitrate_value: ', sum(pop*val for pop, val in county_nitrate_list) / sum(pop for pop, val in county_nitrate_list))

[(36010, 4.22), (77, 6.78), (1000, 0.011), (1200, 0.011), (1500, 4.22), (11678, 0.011), (961, 1.9), (986, 0.011), (2005, 0.011), (651, 0.011), (130, 0.169), (1965, 0.48), (70, 0.2), (58737, 4.88), (2050, 1.5), (136977, 0.48), (5078, 1.5), (178250, 0.15)]

Adams County, Colorado - mean_nitrate_value:  1.255624127923519
